In [8]:
import pickle

from train_model import train_step, test_step
from utils.load_data import get_data
from utils.make_dict import train_bow, get_bow

In [9]:
args ={'dataset': 'MNIST',
       'dataroot': './data',
       'model': 'custom_SVM',
       'kernel': 'gaussian',
       'validation': 0.1,
       'C': 5.0,
       'sigma': 1.0,
       'batch': 1000,
       'dict_size': 100,
       'train': True,
       'load_cluster': False,
       'cuda': False,
       'depth': 100,
       'forest': 10,
       'bag_size': 10000
       }

In [10]:
trainX, trainy = get_data(dataset=args['dataset'], train=True, dataroot=args['dataroot'])

if args['dataset'] == 'cifar10':
    trainX = trainX.reshape((-1, 32, 32, 3), order='F')

if args['load_cluster']:
    with open("./cluster.dump", "rb") as f:
        cluster = pickle.load(f)
else:
    cluster = train_bow(trainX, num_dict=args['dict_size'], num_select=10000)
    with open("./cluster.dump", "wb") as f:
        pickle.dump(cluster, f)

trainFeature = get_bow(trainX, cluster, num_dict=args['dict_size'])

In [11]:
if args['model'] == 'custom_SVM' or args['model'] == 'sklearn_SVM':
    hyper_C = [0.5, 1.0, 5.0]
    hyper_sigma = [0.5, 1.0, 5.0] 

    best_C = None
    best_sigma = None
    best_valid = 0.0

    for C in hyper_C:
        for sigma in hyper_sigma:
            # Test hyperparameter
            args['C'] = C
            args['sigma'] = sigma

            # Get result
            _, train_acc_list, valid_acc_list  = \
                train_step(args, trainFeature, trainy)

            # Evaluation parameter
            tra = sum(train_acc_list) / len(train_acc_list)
            val = sum(valid_acc_list) / len(valid_acc_list)

            if val > best_valid:
                best_valid = val
                best_C = C
                best_sigma = sigma

            # Print result
            print("C: %f Sigma: %f Train accuracy: %f Valid accuracy: %f"%(C, sigma, tra, val))

    print("Best C: %f Best sigma: %f"%(best_C, best_sigma))

    args['C'] = best_C
    args['sigma'] = best_sigma
    args['part'] = False
    models, train_acc_list, valid_acc_list = \
                train_step(args, trainFeature, trainy)
else:
    _, train_acc_list, valid_acc_list  = \
                train_step(args, trainFeature, trainy)

100%|██████████| 10/10 [00:44<00:00,  4.44s/it]


C: 0.500000 Sigma: 0.500000 Train accuracy: 100.000000 Valid accuracy: 50.446667


100%|██████████| 10/10 [00:43<00:00,  4.34s/it]


C: 0.500000 Sigma: 1.000000 Train accuracy: 100.000000 Valid accuracy: 50.876667


100%|██████████| 10/10 [00:46<00:00,  4.65s/it]


C: 0.500000 Sigma: 5.000000 Train accuracy: 100.000000 Valid accuracy: 55.616667


 30%|███       | 3/10 [00:16<00:39,  5.59s/it]


KeyboardInterrupt: 

In [ ]:
testX, testy = get_data(dataset=args['dataset'], train=False, dataroot=args['dataroot'])
if args['dataset'] == 'cifar10':
    testX = testX.reshape((-1, 32, 32, 3), order='F')
testFeature = get_bow(testX, cluster, num_dict=args['dict_size'])

In [ ]:
test_acc_list, test_prec_list, test_recall_list, test_f1_list = test_step(args, testFeature, testy, models)

In [ ]:
print("mA:", sum(test_acc_list) / len(test_acc_list))
print("mP:", sum(test_prec_list) / len(test_prec_list))
print("mR:", sum(test_recall_list) / len(test_recall_list))
print("mF1:", sum(test_f1_list) / len(test_f1_list))